In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np

from utils.plotting import overlaySegment, countParameters, flow2img
from utils.layers import warp, warpImage
from utils.encoding import dice_coeff, labelMatrixOneHot

In [ ]:
baseline = cv2.optflow.DualTVL1OpticalFlow_create()

In [ ]:
img = torch.load('Tests/img.pth') 
seg = torch.load('Tests/seg.pth')

fix = 3; mov = 12;
fixed = img[fix:fix+1,:,:].float() /255
moving = img[mov:mov+1,:,:].float() /255

fixed_seg = seg[fix:fix+1, :,:].long().contiguous()
moving_seg = seg[mov:mov+1,:,:].long().contiguous()

C,H,W = fixed.shape

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.title("Fixed scann")
plt.imshow(fixed.squeeze())
plt.subplot(232)
plt.title("Fixed segmentation")
plt.imshow(fixed_seg.squeeze())
plt.subplot(233)
plt.title("Overlayed for fixed frame")
plt.imshow(overlaySegment(fixed.squeeze(),fixed_seg.squeeze(), False).numpy())
plt.subplot(234)
plt.title("Moving scan")
plt.imshow(moving.squeeze())
plt.subplot(235)
plt.title("Moving segmentation")
plt.imshow(moving_seg.squeeze())
plt.subplot(236)
plt.title("Overlay moving frame")
plt.imshow(overlaySegment(moving.squeeze(),moving_seg.squeeze(),False).numpy())
plt.show()

### Check baseline Flow

In [ ]:
in1 = fixed.view(H,W,1).numpy().astype(np.float32) / 255
in2 = moving.view(H,W,1).numpy().astype(np.float32) / 255

In [ ]:
flow = baseline.calc(in1,in2,None)

In [ ]:
plt.imshow(flow2img(flow))

In [ ]:
torch.from_numpy(flow).view(2,H,W).unsqueeze(0).shape

In [ ]:
moving_seg.unsqueeze(0).shape

In [ ]:
#warped_seg = warp(moving_seg.float().unsqueeze(0).cuda(), torch.from_numpy(flow).view(2,H,W).unsqueeze(0).cuda()).cpu().to(torch.int64)
warped_seg = warpImage(moving_seg.view(1,1,H,W).float(), torch.from_numpy(flow).view(1,2,H,W))

In [ ]:
warped_seg.shape

In [ ]:
d0 = dice_coeff(moving_seg,fixed_seg,8)
print(d0, d0.mean())

In [ ]:
d1 = dice_coeff(warped_seg.unsqueeze(0), fixed_seg,8)
print(d1, d1.mean())

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplot(131)
plt.title('Warped')
plt.imshow(overlaySegment(fixed.squeeze(),warped_seg.squeeze() ,False).numpy())
plt.subplot(132)
plt.title('fixed')
plt.imshow(overlaySegment(fixed.squeeze(),fixed_seg.squeeze() ,False).numpy())
plt.subplot(133)
plt.title('moving')
plt.imshow(overlaySegment(moving.squeeze(),moving_seg.squeeze(),False).numpy())

### Verdict
- baseline already implemented
- sadly not finetunable.
- Maybe gridsearch for hyper params, on the validation dataset.
- good performance